In [1]:
from doculens.models import EmbeddingModel


embedding_model = EmbeddingModel()
embedding_model.config

/home/octoopt/anaconda3/envs/rag/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/octoopt/anaconda3/envs/rag/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


ModelConfig(device='cuda', model_name='keepitreal/vietnamese-sbert')

In [3]:
sample = "Hiệp hội Công nghiệp ghi âm Việt Nam hoạt động trong những lĩnh vực nào?"


embedding_model.embed(sample)

array([[ 7.31994910e-03,  1.40865996e-01, -9.59775934e-04,
        -2.10210428e-01,  4.22635943e-01,  6.65427983e-01,
        -1.98055968e-01,  1.92148402e-01, -1.72215283e-01,
         2.17930466e-01,  1.59431070e-01,  1.43577099e-01,
         6.04931414e-01, -2.22751265e-03, -2.78937947e-02,
         2.19161227e-01,  3.12417269e-01, -8.97035934e-03,
         5.70143759e-02, -1.71511531e-01,  2.14563519e-01,
         4.49207008e-01,  2.71661192e-01,  1.73510775e-01,
        -2.17540666e-01,  2.85483450e-01,  3.42878044e-01,
         1.03309810e-01, -1.75041854e-01,  4.61441189e-01,
        -3.35438967e-01,  2.56961882e-01,  1.69644877e-01,
        -7.23919272e-01,  2.82167271e-03,  6.33383915e-02,
        -1.51719272e-01,  5.78777254e-01,  1.31913945e-01,
         2.10182101e-01,  1.37827754e-01, -2.39302337e-01,
         7.96478748e-01,  5.08418024e-01, -3.90665054e-01,
         2.97375828e-01,  4.39052135e-02, -5.29473543e-01,
         5.33862233e-01,  1.80093974e-01, -4.48313355e-0

# Setup MilvusDB

In [3]:
import pandas as pd

from doculens.config import DatasetConfig


ds_conf = DatasetConfig()
df = pd.read_csv(ds_conf.corpus_dir)
df.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",4


In [ ]:
from doculens.retriever import DoculensRetreiver
from doculens.models import EmbeddingModel


embedding_model = EmbeddingModel()

retriever = DoculensRetreiver(embedding_model=embedding_model)

# Inference

In [7]:
result = retriever.retrieve("Hiệp hội Công nghiệp ghi âm Việt Nam là tổ chức gì?")

['question', 'context', 'cid']


In [15]:
for res in result[0]: 
    print(res['entity']['context'])
    print(res['entity']['cid'])

Tôn chỉ, mục đích\n1. Hội Kiến tạo Việt Nam (sau đây gọi là Hội) là một tổ chức xã hội - nghề nghiệp tự nguyện của những công dân Việt Nam hoạt động trong chuyên ngành Kiến tạo. Hội Kiến tạo Việt Nam là Hội thành viên của Tổng Hội Địa chất Việt Nam. Hội hoạt động trong khuôn khổ pháp luật Nhà nước Cộng hòa xã hội chủ nghĩa Việt Nam và theo Điều lệ Hội.\n2. Mục đích của Hội là tập hợp đoàn kết rộng rãi những người hoạt động trong chuyên ngành Kiến tạo nhằm giúp đỡ nhau phát huy tài năng và trí tuệ, nâng cao trình độ chuyên môn, đạo đức nghề nghiệp cống hiến cho sự nghiệp công nghiệp hóa và hiện đại hóa đất nước góp phần xây dựng nước Việt Nam giàu mạnh xã hội công bằng dân chủ văn minh.
168015
Địa vị pháp lý của Hiệp hội công chứng viên Việt Nam\n1. Hiệp hội công chứng viên Việt Nam (sau đây gọi tắt là Hiệp hội) là tổ chức xã hội - nghề nghiệp toàn quốc của các công chứng viên Việt Nam, có tư cách pháp nhân, có con dấu và tài khoản riêng.\n2. Tên giao dịch quốc tế của Hiệp hội bằng tiến

# Evaluate

In [4]:
corpus_df = pd.read_csv(ds_conf.corpus_dir)
test_df = pd.read_csv(ds_conf.public_test_dir)

In [5]:
len(test_df)

10000

In [6]:
test_df.columns

Index(['question', 'qid'], dtype='object')

In [ ]:
from doculens.helpers import process_data_in_batches


result_dict = {
    'question': [], 
    'qid': [],
    'context': [], 
    'cid': []
}


for batch in process_data_in_batches(test_df, batch_size=1000): 

    # Iterate via each instance in batch
    for idx in range(len(batch)): 
        question = batch.iloc[idx]['question']
        qid = batch.iloc[idx]['qid']
        
        # Retrieve relevant context
        contexts = []
        cids = []
        
        try: 
            result = retriever.retrieve(question)
            for res in result[0]: 
                res_entity = res['entity']
                contexts.append(res_entity['context'])
                cids.append(res_entity['cid'])
        except:
            contexts = [None]
            cids = [-1]
            
        result_dict['question'].append(question)
        result_dict['qid'].append(qid)
        result_dict['context'].append(contexts)
        result_dict['cid'].append(cids)

['question', 'context', 'cid']
Hiệp hội Công nghiệp ghi âm Việt Nam hoạt động trong những lĩnh vực nào?
['question', 'context', 'cid']
Báo cáo nghiên cứu khả thi đầu tư xây dựng là tài liệu trình bày về các nội dung gì?
['question', 'context', 'cid']
Lịch khai giảng năm học 2022 - 2023 đối với học sinh giáo dục mầm non, giáo dục phổ thông và giáo dục thường xuyên là khi nào?
['question', 'context', 'cid']
Số định danh cá nhân có được dùng thay thế các giấy tờ liên quan đến nhân thân khi thực hiện các thủ tục liên quan đến nhà ở không?
['question', 'context', 'cid']
Trợ cấp đối với Chủ tịch Hội cựu chiến binh cấp xã nghỉ hưu, thôi làm công tác Hội được tính thế nào?
['question', 'context', 'cid']
Mẫu đơn ủy quyền nhận trợ cấp thất nghiệp theo quy định là mẫu gì?
['question', 'context', 'cid']
Sĩ quan quân đội có được nghỉ phép hàng tuần hay không?
['question', 'context', 'cid']
Cây xăng có được phép tuyển lao động nữ mang thai vào làm công việc bán lẻ xăng dầu tại đây không?
['question'

In [21]:
pd.DataFrame(result_dict).to_csv('./result.csv')